# STARDUST interactive

This Jupyter Notebook provides an interactive version of STARDUST signal extraction. Functions are defined and imported from util.py. Use help() to see the documentation. For a more detailed description of the STARDUST pipeline, please see the biorxiv paper and github page for more information. 

## 1. Environment set up

In [ ]:
import os, scipy, sys, pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt
from src.STARDUST.util import * 

## 2. Data input

Input paths and parameters. Find input file paths using find_files(). Note that the current code assumes that the input time-series data is stored in a csv, and ends with "signals.csv", the ROA mask is stored as a tiff file with keyword "ROA", and similarly the cell mask is a tiff file with keyword "cell". 

You can also directly input the paths. 

In [ ]:
# input input and output directories
input_dir = input('Enter the path to the folder containing the input files: ')
output_dir = input('Enter the path to the folder containing the output files: ')

# input metadata related to the experiment
keyword = input('Enter the common keyword in input file (csv, ROA mask and cell mask) names: ')
drug_frame = float(input('Enter the frame number of drug application (enter 0 if no drug was applied): '))
frame_rate = float(input("Enter the frame rate (in Hz) of the recording (e.g. 1): "))
spatial_resolution = float(input('Enter the spatial resolution of the recording (in µm/pixel, if one pixel size is 0.884 x 0.884 µm^2, enter 0.884): '))
output_filename = input('Enter the output file name: ')
output_path = os.path.join(output_dir, output_filename)

# find the input files using find_files function
csv_path, ROA_mask_path, cell_mask_path = find_files(input_dir, keyword)

Next, read in ROA mask and cell mask. 

In [ ]:
ROA_map_array, ROA_map_labeled, ROA_map_count = read_tif(ROA_mask_path, "ROA")
cell_map_array, cell_map_labeled, cell_count = read_tif(cell_mask_path, "cell")

visualize_map(ROA_map_array = ROA_map_array)

## 3. Signal preprocessing

In [ ]:
# Smooth traces using a filter
raw_traces, filtered_traces = raw_to_filtered(csv_path)

In [ ]:
ROA_count, frame_count = check_traces(filtered_traces)

### Optinal: Signal correction using linear regression

This optional step uses the correct_shift() function to detect and correct gradual linear drift in the traces. Ideally, the slope distribution histogram should roughly center around zero. If the distribution is not centered around zero, it might indicate photobleaching or a significant z drift during the recording. 

In [ ]:
# optional: correct traces for shift using linear regression
corrected_traces, reg = correct_shift(filtered_traces)

## 4. Baseline determination and signal detection

In [ ]:
help(iterative_baseline)

In [ ]:
# baseline determination
dff_traces, baselines, thresholds, signal_frames, signal_boundaries, signal_threshold = iterative_baseline(corrected_traces)

## Checkpoint 1: dF/F traces heatmap

Visualze dF/F traces using heatmap. Each row represents one ROA and each column represents one frame.  

Note that for heatmap coloring, *vmin* is set at 0, and *vmax* is set at (signal_threshold + 2) * average thresholds across all ROAs. For example, if the signal_threshold is set at 3SD, any transients that has a dF/F value above baseline + 5SD will be colored red to facilitate visualization. You can adjust the vmax parameter if needed. 

In [ ]:
sns.heatmap(dff_traces, vmin = 0, vmax = (signal_threshold + 2) * thresholds.mean(), 
            xticklabels=100, yticklabels= False, cmap = 'jet', square = True);

## 5. Signal feature extraction

In [ ]:
signal_stats = analyze_signal(dff_traces, signal_frames, signal_boundaries, frame_rate, drug_frame)

In [ ]:
signal_stats.head()

## 6. ROA and cell assignment

In [ ]:
# add corresponding cell ID to the signal stats
df_ROA_cell = align_ROA_cell(ROA_map_labeled, cell_map_labeled, ROA_map_count, spatial_resolution)
signal_stats = pd.merge(df_ROA_cell, signal_stats, on = 'ROA_ID', how = 'right')

In [ ]:
signal_stats.head(10)

### ROA based analysis

Note that ROAs that do not have any cell assignment is listed as cell 0 for cell ID. 

In [ ]:
ROA_based, df_ROA_cell = ROA_analysis(signal_stats, df_ROA_cell, frame_count, frame_rate, drug_frame)

In [ ]:
ROA_based.head()

### Cell based analysis

Note that ROAs that do not have any cell assignment is listed as cell 0 for cell ID. This cell should be omitted in later analysis. 

In [ ]:
cell_based = cell_analysis(signal_stats, df_ROA_cell)

In [ ]:
cell_based.head()

## Checkpoint 2: ROA stats

In [ ]:
sns.histplot(data = ROA_based, x = 'frequency_permin');

In [ ]:
ROA_summary = df_ROA_cell.groupby('ROA_type').count()
ROA_summary

## Checkpoint 3: Individual traces
Use the inspect_trace() function to visualize traces. 

### Visualize inactive ROAs
The following example checks all ROAs that are "inactive" based on our pipeline but was intially determined as active ROA by AQuA. 

In [ ]:
inactive_ROAs = df_ROA_cell[df_ROA_cell['ROA_type'] == 'inactive']['ROA_ID'].to_list()
inspect_trace(inactive_ROAs, dff_traces, baselines, thresholds, drug_frame)

### Visualize ROAs with large slope during optional correction

In [ ]:
check_ROAs = pull_largeslope_traces(ROA_count, reg)
inspect_trace(check_ROAs, dff_traces, baselines, thresholds, drug_frame)

### Visualize a few randomly selected ROAs

In [ ]:
random_ROAs = np.random.choice(ROA_count, 10)
random_ROAs.sort()
inspect_trace(random_ROAs, dff_traces, baselines, thresholds, drug_frame)

## 7. Data output

In [ ]:
metadata = metadata_todf()
output_data(save_as = 'csv')